In [60]:
import pickle
import re
import pdb

import pandas as pd
import numpy as np

from IPython.display import display

### Converting Raw API data to DataFrame

In [166]:
data = pd.read_json('data/raw_api_data.txt')

cols_to_parse = ['Gender', 'SeverityType', 'Locale', 'ProductCategory']
cols_to_add = [['GenderDescription','GenderId','GenderPublicName'],
['IncidentDetails','SeverityTypeDescription','SeverityTypePublicName'],
['LocaleDescription','LocalePublicName'],
['ProductCategoryDescription','ProductCategoryPublicName']]
new_df = pd.DataFrame()
cols_dict = dict(zip(cols_to_parse, cols_to_add))

for key in cols_dict:
    placeholder = pd.DataFrame(columns = cols_dict[key])
    for (i, row) in data.iterrows():
        e = row[key]
        value_holder = []
        for item in cols_dict[key]:
            try:
                component = e[item]
            except:
                component = 'Missing'
            value_holder.append(component)
        placeholder.loc[i, :] = value_holder
    if new_df.shape[0] == 0:
        new_df = placeholder
    else:
        new_df = pd.concat([new_df, placeholder], axis=1)
        
new_df2 = pd.concat([data, new_df], axis=1)
new_df2 = new_df2.drop(['CompanyComments', 'Gender','IncidentDocuments','IncidentDetails', 'Locale', 'ProductCategory',
                      'RelationshipType','SeverityType', 'SourceType'], axis=1)
new_df2.to_pickle('data/cleaned_api_data')

### Analysis

In [3]:
data = pickle.load( open("data/cleaned_api_data", "rb" ) )
neiss = pd.read_csv('data/NEISS-data-2015-updated-APRIL2016.csv')

In [31]:
products = data['ProductCategoryDescription'].drop_duplicates()
for product in products:
    print(product)

Hair Curlers, Curling Irons, Clips & Hairpins
Cribs
Electric Ranges or Ovens (Excl Counter-top Ovens)
Refrigerators
Diapers
Pogo Sticks
Coal or Wood-burning Stoves
Televisions
Candles, Candlesticks and Other Candle Holders
Sheets or Pillowcases
Baby Gates or Barriers
Footwear
Clothing Accessories
Batteries
Light Bulbs
Dishwashers
Toy Musical Instruments
Lighted Make-up Mirrors
Infant & Toddler Play Ctrs, Excl Jumpers,bouncers&exercisers
Computers (Equipment and Electronic Games)
Baby Rattles
Air Purifiers
Electric Timers
Bicycles and Accessories, (Excl.mountain or All-terrain)
Chain Saws
Extension Cords
Outerwear
Baby Strollers
Ranges or Ovens, Not Specified
Electric Can Openers
Gas Ranges or Ovens
Electric Coffee Makers or Teapots
Other Ranges or Ovens
Dolls, Plush Toys, and Action Figures
Electric Grills
Snow Throwers, Blowers
Coffee Makers or Teapots, Not Specified
Day Wear
Electric Heaters, Not Elsewhere Classified
Microwave Ovens
Mattresses, Not Specified
Pacifiers or Teething Rin

In [72]:
def product_parser(item):
    separators = [',', '&', '(']
    if any(x in item for x in separators):
        separators_used = [x for x in separators if x in item]
        parsed_items = []
        for separator in separators_used:
            if separator in [',', '&']:
                separated = item.split(separator)
                parsed_items.append(separated)
                return [item for sublist in parsed_items for item in sublist if not any(x in item for x in [',', '&'])]
            elif separator == '(':
                separated = item.split(separator)[0]
                parsed_items.append(separated)
                return parsed_items
    else:
        return item

def and_or_splitter(split_string, item):
    if split_string not in ['and', 'or']:
        return item
    else:
#         set_trace()
        splitted = item.split(split_string)
        if len(splitted[0].split(' ')) > 1:
            item_split = splitted[0].split(' ')[:-1]
            final_item = (' ').join(item_split) + ' ' + splitted[-1]
        return [splitted[0], final_item]

def product_cleaner(list_of_items):
    final_list = []
    if isinstance(list_of_items, str):
        and_splitted = and_or_splitter('and', list_of_items)
        return and_or_splitter('or', and_splitted)
    for item in list_of_items:
        and_splitted = and_or_splitter('and', item)
        final_list.append(and_or_splitter('or', and_splitted))
    return final_list



In [37]:
'Art or Ceramics Supplies and Equipment (Excl. Paintbrushes)'
'Book Bags or Back Carriers (Exc. Baby Carr. Luggage, Cam'
'Liquid Room Deodorizers or Fresheners'
'Candles, Candlesticks and Other Candle Holders'
'Ranges or Ovens, Not Specified'

True